# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/cities.csv"
cities_df = pd.read_csv(file_path)


In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,beringovskiy,63.05,179.32,50.41,76,0,12.15,RU,1595466878
1,rikitea,-23.12,-134.97,66.94,65,100,14.67,PF,1595466759
2,durant,33.99,-96.37,87.80,58,1,12.75,US,1595466641
3,punta arenas,-53.15,-70.92,26.60,92,57,4.70,CL,1595466722
4,salalah,17.02,54.09,78.80,94,90,11.41,OM,1595466865
...,...,...,...,...,...,...,...,...,...
518,kasulu,-4.58,30.10,66.20,77,0,3.49,TZ,1595467021
519,santiago del estero,-27.80,-64.26,59.65,46,100,3.62,AR,1595466878
520,derzhavinsk,51.10,66.32,68.32,66,100,18.66,KZ,1595466845
521,pevek,69.70,170.31,55.40,53,98,10.40,RU,1595467022


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
new_york_coordinates = (40.75, -74.00)

fig = gmaps.figure(center=new_york_coordinates, zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(cities_df[["Lat", "Lng"]], weights=cities_df["Humidity"])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = cities_df[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 10)]
ideal_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,surt,31.21,16.59,75.76,76,0,7.52,LY,1595466881
29,tartus,34.89,35.89,75.20,78,0,4.70,SY,1595466885
68,batagay-alyta,67.80,130.41,72.86,46,0,5.91,RU,1595466894
104,arraial do cabo,-22.97,-42.03,71.38,86,0,7.25,BR,1595466906
110,tomino,41.99,-8.76,71.60,68,0,0.60,ES,1595466909
115,abu kamal,34.45,40.92,79.63,29,0,6.31,SY,1595466910
133,fasa,28.94,53.65,73.40,43,0,4.70,IR,1595466916
195,forio,40.73,13.86,79.00,64,0,4.99,IT,1595466937
200,suez,29.97,32.53,72.59,81,0,7.63,EG,1595466670
227,vila velha,-20.33,-40.29,75.99,78,0,8.05,BR,1595466794


In [6]:
index_ideal = ideal_cities_df.reset_index(drop=True)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
index_ideal = ideal_cities_df.reset_index(drop=True)

In [9]:
index_ideal["hotel_name"] = ""

In [10]:
for index, row in index_ideal.iterrows():
    
    lat_var2 = row["Lat"]
    lng_var2 = row["Lng"]
    
    target_coordinates1 = f"{lat_var2}, {lng_var2}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"
    fields_search = "formatted_address"

    params = {
        "location": target_coordinates1,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key,
    }
    
    try:
        response = requests.get(base_url, params=params).json()
        index_ideal.loc[index, "hotel_name"] = response["results"][0]["name"]
        
    except:
        print("no hotel in vicinity")

no hotel in vicinity


In [11]:
clean_ideal = index_ideal[index_ideal["hotel_name"] != ""]
clean_ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name
0,surt,31.21,16.59,75.76,76,0,7.52,LY,1595466881,City Hotel
1,tartus,34.89,35.89,75.20,78,0,4.70,SY,1595466885,Grand Hotel Tartus
2,batagay-alyta,67.80,130.41,72.86,46,0,5.91,RU,1595466894,Batagay-Alyta
3,arraial do cabo,-22.97,-42.03,71.38,86,0,7.25,BR,1595466906,Hotel da Canoa
4,tomino,41.99,-8.76,71.60,68,0,0.60,ES,1595466909,Hotel Minho Belo
6,fasa,28.94,53.65,73.40,43,0,4.70,IR,1595466916,Jahangardi Hotel
7,forio,40.73,13.86,79.00,64,0,4.99,IT,1595466937,San Montano Resort & SPA
8,suez,29.97,32.53,72.59,81,0,7.63,EG,1595466670,LUSINDA HOTEL
9,vila velha,-20.33,-40.29,75.99,78,0,8.05,BR,1595466794,Sheraton Vitoria Hotel
10,cururupu,-1.83,-44.87,73.47,89,0,2.89,BR,1595466957,Hotel Cururupu


In [12]:
clean_index = clean_ideal.reset_index(drop=True)
clean_index

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel_name
0,surt,31.21,16.59,75.76,76,0,7.52,LY,1595466881,City Hotel
1,tartus,34.89,35.89,75.20,78,0,4.70,SY,1595466885,Grand Hotel Tartus
2,batagay-alyta,67.80,130.41,72.86,46,0,5.91,RU,1595466894,Batagay-Alyta
3,arraial do cabo,-22.97,-42.03,71.38,86,0,7.25,BR,1595466906,Hotel da Canoa
4,tomino,41.99,-8.76,71.60,68,0,0.60,ES,1595466909,Hotel Minho Belo
5,fasa,28.94,53.65,73.40,43,0,4.70,IR,1595466916,Jahangardi Hotel
6,forio,40.73,13.86,79.00,64,0,4.99,IT,1595466937,San Montano Resort & SPA
7,suez,29.97,32.53,72.59,81,0,7.63,EG,1595466670,LUSINDA HOTEL
8,vila velha,-20.33,-40.29,75.99,78,0,8.05,BR,1595466794,Sheraton Vitoria Hotel
9,cururupu,-1.83,-44.87,73.47,89,0,2.89,BR,1595466957,Hotel Cururupu


In [14]:
locations_df = clean_index.drop(['Max Temp', 'Humidity', "Cloudiness", "Wind Speed", "Date"], axis=1)

In [15]:
locations_df['location'] = list(zip(locations_df.Lat, locations_df.Lng))
locations_df

,City,Lat,Lng,Country,hotel_name,location
0,surt,31.21,16.59,LY,City Hotel,"(31.21, 16.59)"
1,tartus,34.89,35.89,SY,Grand Hotel Tartus,"(34.89, 35.89)"
2,batagay-alyta,67.80,130.41,RU,Batagay-Alyta,"(67.8, 130.41)"
3,arraial do cabo,-22.97,-42.03,BR,Hotel da Canoa,"(-22.97, -42.03)"
4,tomino,41.99,-8.76,ES,Hotel Minho Belo,"(41.99, -8.76)"
5,fasa,28.94,53.65,IR,Jahangardi Hotel,"(28.94, 53.65)"
6,forio,40.73,13.86,IT,San Montano Resort & SPA,"(40.73, 13.86)"
7,suez,29.97,32.53,EG,LUSINDA HOTEL,"(29.97, 32.53)"
8,vila velha,-20.33,-40.29,BR,Sheraton Vitoria Hotel,"(-20.33, -40.29)"
9,cururupu,-1.83,-44.87,BR,Hotel Cururupu,"(-1.83, -44.87)"


In [16]:
locations_dict = locations_df.to_dict("records")
locations_dict

[{'City': 'surt',
  'Lat': 31.21,
  'Lng': 16.59,
  'Country': 'LY',
  'hotel_name': 'City Hotel',
  'location': (31.21, 16.59)},
 {'City': 'tartus',
  'Lat': 34.89,
  'Lng': 35.89,
  'Country': 'SY',
  'hotel_name': 'Grand Hotel Tartus',
  'location': (34.89, 35.89)},
 {'City': 'batagay-alyta',
  'Lat': 67.8,
  'Lng': 130.41,
  'Country': 'RU',
  'hotel_name': 'Batagay-Alyta',
  'location': (67.8, 130.41)},
 {'City': 'arraial do cabo',
  'Lat': -22.97,
  'Lng': -42.03,
  'Country': 'BR',
  'hotel_name': 'Hotel da Canoa',
  'location': (-22.97, -42.03)},
 {'City': 'tomino',
  'Lat': 41.99,
  'Lng': -8.76,
  'Country': 'ES',
  'hotel_name': 'Hotel Minho Belo',
  'location': (41.99, -8.76)},
 {'City': 'fasa',
  'Lat': 28.94,
  'Lng': 53.65,
  'Country': 'IR',
  'hotel_name': 'Jahangardi Hotel',
  'location': (28.94, 53.65)},
 {'City': 'forio',
  'Lat': 40.73,
  'Lng': 13.86,
  'Country': 'IT',
  'hotel_name': 'San Montano Resort & SPA',
  'location': (40.73, 13.86)},
 {'City': 'suez',
  

In [17]:
hotel_locations = [hotel['location'] for hotel in locations_dict]
info_box_template1 = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template1.format(**hotel) for hotel in locations_dict]

hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add marker layer ontop of heat map
# Display figure
fig2 = gmaps.figure(center=new_york_coordinates, zoom_level=2)

fig2.add_layer(heatmap_layer)
fig2.add_layer(hotel_layer)

fig2

Figure(layout=FigureLayout(height='420px'))